In [1]:
import numpy as np

In [2]:

def interpolasi_data_hari_waktu(data, hari, waktu):
    """
    Mengisi data yang hilang menggunakan interpolasi linier berdasarkan hari dan waktu.
    
    Parameters:
        data (dict): Data yang berisi informasi lalu lintas harian. 
                     Format: {'Senin': {'08:00': {'motor': X, 'mobil': Y, 'truk': Z}, ...}, ...}
        hari (str): Nama hari yang akan diisi datanya.
        waktu (str): Waktu yang akan diisi datanya dalam format HH:MM.
        
    Returns:
        dict: Data interpolasi untuk motor, mobil, dan truk jika data ada di sekitar waktu yang hilang.
    """
    times = []
    motor_values = []
    mobil_values = []
    truk_values = []

    # Konversi waktu ke menit untuk kemudahan perhitungan
    waktu_dalam_menit = lambda w: int(w.split(":")[0]) * 60 + int(w.split(":")[1])

    # Kumpulkan waktu dan data kendaraan yang tersedia
    for w, kendaraan in data.get(hari, {}).items():
        if kendaraan is not None:
            times.append(waktu_dalam_menit(w))
            motor_values.append(kendaraan['motor'])
            mobil_values.append(kendaraan['mobil'])
            truk_values.append(kendaraan['truk'])

    if len(times) < 2:
        # Jika tidak cukup data untuk interpolasi, kembalikan None
        return {"motor": None, "mobil": None, "truk": None}

    # Urutkan data berdasarkan waktu
    sorted_indices = np.argsort(times)
    times = np.array(times)[sorted_indices]
    motor_values = np.array(motor_values)[sorted_indices]
    mobil_values = np.array(mobil_values)[sorted_indices]
    truk_values = np.array(truk_values)[sorted_indices]

    # Gunakan interpolasi linier untuk menghitung nilai yang hilang
    waktu_target = waktu_dalam_menit(waktu)
    motor_interp = np.interp(waktu_target, times, motor_values)
    mobil_interp = np.interp(waktu_target, times, mobil_values)
    truk_interp = np.interp(waktu_target, times, truk_values)

    return {"motor": motor_interp, "mobil": mobil_interp, "truk": truk_interp}

# Contoh penggunaan fungsi
data_lalu_lintas = {
    'Senin': {'07:00': {'motor': 110, 'mobil': 45, 'truk': 8},
              '08:00': {'motor': 120, 'mobil': 50, 'truk': 10},
              '10:00': {'motor': 130, 'mobil': 55, 'truk': 12}},
    'Selasa': {'07:00': {'motor': 105, 'mobil': 43, 'truk': 9},
               '08:00': {'motor': 115, 'mobil': 52, 'truk': 11},
               '10:00': {'motor': 125, 'mobil': 58, 'truk': 14}},
    # Data hari lainnya...
}

hasil = interpolasi_data_hari_waktu(data_lalu_lintas, 'Senin', '08:30')
print(hasil)

hasil = interpolasi_data_hari_waktu(data_lalu_lintas, 'Senin', '09:00')
print(hasil)

hasil = interpolasi_data_hari_waktu(data_lalu_lintas, 'Senin', '09:30')
print(hasil)

{'motor': 122.5, 'mobil': 51.25, 'truk': 10.5}
{'motor': 125.0, 'mobil': 52.5, 'truk': 11.0}
{'motor': 127.5, 'mobil': 53.75, 'truk': 11.5}


In [4]:
import pandas as pd
import numpy as np

# Membuat DataFrame contoh
data = {
    'BULAN': [1, 1, 1, 1, 1, 1, 1],
    'TANGGAL': [1, 1, 1, 1, 1, 1, 1],
    'HARI': ['Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday'],
    'JAM': ['00:00', '00:15', '00:30', '00:45', '01:00', '01:15', '01:30'],
    'MOTOR': [103, np.nan, np.nan, 76, 73, np.nan, np.nan],
    'MOBIL': [25, np.nan, np.nan, 24, 24, np.nan, np.nan],
    'TRUK/BUS': [14, np.nan, np.nan, 11, 7, np.nan, np.nan],
    'JUMLAH': [142, np.nan, np.nan, 111, 104, np.nan, np.nan],
    'SMP': [84, np.nan, np.nan, 68, 62, np.nan, np.nan]
}

df = pd.DataFrame(data)

# Mengisi nilai NaN dengan interpolasi (linear)
df['MOTOR'] = df['MOTOR'].interpolate(method='linear')
df['MOBIL'] = df['MOBIL'].interpolate(method='linear')
df['TRUK/BUS'] = df['TRUK/BUS'].interpolate(method='linear')
df['JUMLAH'] = df['JUMLAH'].interpolate(method='linear')
df['SMP'] = df['SMP'].interpolate(method='linear')

# Mengisi nilai NaN yang tersisa dengan forward fill (ffill)
df['MOTOR'] = df['MOTOR'].fillna(method='ffill')
df['MOBIL'] = df['MOBIL'].fillna(method='ffill')
df['TRUK/BUS'] = df['TRUK/BUS'].fillna(method='ffill')
df['JUMLAH'] = df['JUMLAH'].fillna(method='ffill')
df['SMP'] = df['SMP'].fillna(method='ffill')

# Jika masih ada NaN di awal, gunakan backward fill (bfill)
df['MOTOR'] = df['MOTOR'].fillna(method='bfill')
df['MOBIL'] = df['MOBIL'].fillna(method='bfill')
df['TRUK/BUS'] = df['TRUK/BUS'].fillna(method='bfill')
df['JUMLAH'] = df['JUMLAH'].fillna(method='bfill')
df['SMP'] = df['SMP'].fillna(method='bfill')

# Menampilkan DataFrame hasil
print(df)


   BULAN  TANGGAL    HARI    JAM  MOTOR      MOBIL  TRUK/BUS      JUMLAH  \
0      1        1  Sunday  00:00  103.0  25.000000      14.0  142.000000   
1      1        1  Sunday  00:15   94.0  24.666667      13.0  131.666667   
2      1        1  Sunday  00:30   85.0  24.333333      12.0  121.333333   
3      1        1  Sunday  00:45   76.0  24.000000      11.0  111.000000   
4      1        1  Sunday  01:00   73.0  24.000000       7.0  104.000000   
5      1        1  Sunday  01:15   73.0  24.000000       7.0  104.000000   
6      1        1  Sunday  01:30   73.0  24.000000       7.0  104.000000   

         SMP  
0  84.000000  
1  78.666667  
2  73.333333  
3  68.000000  
4  62.000000  
5  62.000000  
6  62.000000  
